In [2]:
!pip install -U dspy
!pip install langchain
!pip install -qU langchain-community faiss-cpu
!pip install --upgrade --quiet  llama-cpp-python

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp -r /content/drive/MyDrive/Colab\ Notebooks/GST-QA/* .
mkdir llama-model/

In [3]:
import os
from google.colab import userdata
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

In [4]:
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="bartowski/Llama-3.2-1B-Instruct-GGUF", filename="Llama-3.2-1B-Instruct-Q6_K.gguf", local_dir='llama-model/')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


'llama-model/Llama-3.2-1B-Instruct-Q6_K.gguf'

In [5]:
from settings import *
import dspy
import json

In [6]:
# Local Anthropic
import dspy
lm = dspy.LM('anthropic/claude-3-opus-20240229')
dspy.configure(lm=lm)

In [ ]:
# Local Ollama
# from litellm import completion

# llama = dspy.LM(
#     "ollama_chat/localllama",
#     model_type="chat",
#     temperature=TEMPRATURE,
#     api_base="http://localhost:11434",
# )
# dspy.configure(lm=llama)

In [ ]:
# Local LLAMA CPP (DEPRECATED: Has issues)

# REF-1: LLAMA CPP: https://dspy.ai/deep-dive/language_model_clients/lm_local_models/LlamaCpp/?h=l
# REF-2: GENERAL RAG:  https://dspy.ai/tutorials/rag/#configuring-the-dspy-environment
# REF-3: https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb

# from llama_cpp import Llama


# llm = Llama(
#     temperature=0.2,
#     model_path=MODEL_PATH,
#     n_ctx=(N_DOCS_RETREIVE+2)*CHUNK_SIZE,
#     n_gpu_layers=-1,
#     n_batch=32,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
#     max_tokens=4098,
#     top_p=1,
#     verbose=False
# )

# llamalm = dspy.LlamaCpp(model="llama", llama_model=llm,  model_type="chat", temperature=TEMPRATURE)
# dspy.configure(lm=llamalm)

In [7]:
# Loading Data
with open(QUESTION_FILE) as q_f:
    questions = json.load(q_f)

# Mapping To DSPy Mapping
questions = [{'question': q['question'], 'response': q['answer']} for q in questions]
print(f"Data {len(questions)} is loaded.")

Data 48 is loaded.


In [8]:
# Preparing Data
import random

data = [dspy.Example(**q).with_inputs('question') for q in questions]
# Splitting Data Train, Dev=2*Train, Test=2*Train
n_data = len(data)
n_train, n_test = int(n_data/5), 2*int(n_data/5)
n_dev = n_data - n_train - n_test

random.Random(0).shuffle(data)
trainset, devset, testset = data[:n_train], data[n_train:n_train+n_test], data[n_train+n_test:n_data]
print(f"Divided {n_data} to \n Train:{len(trainset)} Test: {len(testset)} Dev:{len(devset)}")

Divided 48 to 
 Train:9 Test: 21 Dev:18


In [9]:
# Loading the vector_store of Langchain
from langchain_community.vectorstores import FAISS
from utils import LlamaCppEmbeddingsFix

embeding = LlamaCppEmbeddingsFix(
    model_path=MODEL_PATH, verbose=False,
    n_ctx=(N_DOCS_RETREIVE+2)*CHUNK_SIZE,
)

vector_store = FAISS.load_local(
    VECTOR_STORE,
    embeddings=embeding,
    allow_dangerous_deserialization=True
)
vector_store.load_local(VECTOR_STORE, embeddings=embeding, allow_dangerous_deserialization=True)
print(f"Loaded {vector_store.index.ntotal} Documents")

retriever = vector_store.as_retriever(
    search_type=SEARCH_METHOD,
    search_kwargs={
    'k': N_DOCS_RETREIVE,
    'score_threshold': SCORE_THRESH
})

llama_new_context_with_model: n_ctx_per_seq (12000) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


Loaded 17 Documents


In [10]:
from dspy.evaluate import SemanticF1

# RAG Based
def search(question):
    docs = retriever.invoke(question)
    return "".join(f"Page {d.metadata['page']}\n"+d.page_content for d in docs)

class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question)
        #print("Context Created", context)
        return self.respond(context=context, question=question)

metric = SemanticF1(decompositional=True)
rag = RAG()
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=1, display_progress=True, display_table=2)

In [11]:
# Single Test Run
example = trainset[0]
pred = rag(question=example.question)
score = metric(example, pred)

print("####### Example ######### \n", example)
print("############ Pred ############## \n", pred)
print("############ Score ############## \n", score)

####### Example ######### 
 Example({'question': 'How can you filter the admission inquiry list?', 'response': 'By date, standard, enquiry number, name, or contact number'}) (input_keys={'question'})
############ Pred ############## 
 Prediction(
    reasoning='Based on the information provided in the operational manual, there are two ways to filter the admission inquiry list:\n\n1. Activity Date Wise Report - This allows you to check date wise admission enquiry details by selecting specific dates and clicking the search icon.\n\n2. Follow Up Date Wise Report - This also allows filtering the admission enquiry list by selecting specific follow up dates and clicking the search icon to see the date wise follow up report.\n\nSo in summary, the admission inquiry list can be filtered either by activity date or by follow up date to narrow down the results.',
    response='According to the operational manual, there are two ways to filter the admission inquiry list:\n\n1. Using the Activity Dat

In [ ]:
dspy.inspect_history()

In [12]:
# Raw Performance
evaluate(rag)

Average Metric: 13.81 / 18 (76.7%): 100%|██████████| 18/18 [06:49<00:00, 22.73s/it]

2024/12/03 15:03:22 INFO dspy.evaluate.evaluate: Average Metric: 13.809770638718009 / 18 (76.7%)


,question,example_response,reasoning,pred_response,SemanticF1
0,What is required to save a registration form?,Fill all required details and click Save,"According to the information provided on page 8, to save a registr...","To save a registration form, you need to fill in the registration ...",✔️ [1.000]
1,How do you generate an MIS report?,Select the required report level and click Generate,The given context does not provide information on how to generate ...,"I apologize, the information provided in the given context does no...",


76.72

In [13]:
# Optimize RAG
tp = dspy.MIPROv2(metric=metric, auto="medium", num_threads=1)

optimized_rag = tp.compile(RAG(), trainset=trainset,
                           max_bootstrapped_demos=2, max_labeled_demos=2,
                           requires_permission_to_run=False)

2024/12/03 15:06:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 7

2024/12/03 15:06:39 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2024/12/03 15:06:39 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2024/12/03 15:06:39 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


100%|██████████| 2/2 [00:40<00:00, 20.22s/it]


Bootstrapped 1 full traces after 1 examples for up to 1 rounds, amounting to 2 attempts.
Bootstrapping set 4/19


100%|██████████| 2/2 [00:02<00:00,  1.30s/it]


...

2024/12/03 15:30:10 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'How do you open a new registration form?', 'response': 'Click Registration > New Registration'}) (input_keys={'question'}): litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 7 (0.0%):  86%|████████▌ | 6/7 [00:14<00:02,  2.19s/it]

2024/12/03 15:30:14 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'What are the three steps in the admission process?', 'response': 'Inquiry, Registration, Admission'}) (input_keys={'question'}): litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 7 (0.0%): 100%|██████████| 7/7 [00:17<00:00,  2.57s/it]

2024/12/03 15:30:14 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 7 (0.0%)
2024/12/03 15:30:14 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 5'].
2024/12/03 15:30:14 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [58.96, 67.25, 42.09, 50.16, 56.76, 42.09, 14.29, 50.16, 42.09, 0.0, 0.0, 67.25, 67.25, 67.25, 67.25, 42.09, 0.0, 0.0, 0.0, 0.0, 67.25, 42.09, 42.09, 67.25, 0.0, 0.0]
2024/12/03 15:30:14 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 67.25
2024/12/03 15:30:14 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2024/12/03 15:30:14 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 67.25!


In [14]:
evaluate(optimized_rag)

  0%|          | 0/18 [00:00<?, ?it/s]

2024/12/03 15:30:16 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'What is required to save a registration form?', 'response': 'Fill all required details and click Save'}) (input_keys={'question'}): litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 18 (0.0%):   6%|▌         | 1/18 [00:02<00:35,  2.09s/it]

2024/12/03 15:30:18 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'How do you generate an MIS report?', 'response': 'Select the required report level and click Generate'}) (input_keys={'question'}): litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 18 (0.0%):  11%|█         | 2/18 [00:03<00:31,  1.95s/it]

2024/12/03 15:30:20 ERROR dspy.utils.parallelizer: Error processing item Example({'question': "Where is the 'New Registration' button located?", 'response': 'Under the Registration menu'}) (input_keys={'question'}): litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 18 (0.0%):  17%|█▋        | 3/18 [00:06<00:30,  2.02s/it]

2024/12/03 15:30:22 ERROR dspy.utils.parallelizer: Error processing item Example({'question': 'What do fee structure details include?', 'response': 'Installment details, fee type, and categories'}) (input_keys={'question'}): litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}. Set `provide_traceback=True` to see the stack trace.


Average Metric: 0.00 / 18 (0.0%):  22%|██▏       | 4/18 [00:07<00:27,  1.94s/it]

BadRequestError: litellm.BadRequestError: AnthropicException - {"type":"error","error":{"type":"invalid_request_error","message":"Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits."}}

In [ ]:
optimized_rag.save("optimized_rag.json")

In [ ]:
dspy.inspect_history(n=2)

In [ ]:
!cp optimized_rag.json /content/drive/MyDrive/Colab\ Notebooks/GST-QA/